Code zum Finden von Fehlern in deiner JSON Finetuning Datei

In [ ]:
import json  # Notwendiger Import für JSON-Operationen

dataset = []
data_path = "deine Daten"

# Helper to diagnose JSON decoding errors
with open(data_path, 'r', encoding='utf-8') as f:
    for idx, line in enumerate(f):
        try:
            dataset.append(json.loads(line))
        except json.JSONDecodeError:
            print(f"Error decoding JSON on line {idx + 1}:")
            print(line)
            print("-" * 40)

In [ ]:
import json  # Notwendiger Import für JSON-Operationen
from collections import defaultdict  # Notwendig für defaultdict

data_path = "data/training03_set.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")
